In [5]:
import numpy as np 
import yfinance as yf

### Generate example data

In [6]:
# Pull 1 years worth of dailiy price data for 4 stocks from yahoo finance
tickers = sorted(['AAPL', 'F', 'VZ', 'KO'])
start = '2023-01-01'
end = '2023-12-31'

data = yf.download(tickers, start, end, )

prices = data['Adj Close']

prices.head()

[*********************100%***********************]  4 of 4 completed


Ticker,AAPL,F,KO,VZ
Date,,,,
2023-01-03 00:00:00+00:00,123.768463,9.878720,59.665287,35.039959
2023-01-04 00:00:00+00:00,125.045029,10.157829,59.636856,35.922073
2023-01-05 00:00:00+00:00,123.718971,10.360816,58.954426,36.419891
2023-01-06 00:00:00+00:00,128.271103,10.639923,60.091812,36.847847
2023-01-09 00:00:00+00:00,128.795593,10.732961,59.343029,36.699703


In [7]:
# Turn the raw price data into a daily returns dataframe
returns = prices.pct_change()
returns.head()

Ticker,AAPL,F,KO,VZ
Date,,,,
2023-01-03 00:00:00+00:00,NaN,NaN,NaN,NaN
2023-01-04 00:00:00+00:00,0.010314,0.028254,-0.000477,0.025175
2023-01-05 00:00:00+00:00,-0.010605,0.019983,-0.011443,0.013858
2023-01-06 00:00:00+00:00,0.036794,0.026939,0.019293,0.011751
2023-01-09 00:00:00+00:00,0.004089,0.008744,-0.012461,-0.004020


In [8]:
# Take the average historical return as a expected return vector for testing
# In practice this vector would be computed with signal research
expected_returns = returns.mean()

expected_returns.reset_index().rename(columns={0: 'E[r]'})

,Ticker,E[r]
0,AAPL,0.001835
1,F,0.000822
2,KO,-0.000104
3,VZ,0.000138


In [9]:
# Take the historical convariance matrix of the returns dataframe
# In practice this matrix would be forecasted with signal research
covariance_matrix = returns.cov()

covariance_matrix

Ticker,AAPL,F,KO,VZ
Ticker,,,,
AAPL,0.000158,0.000068,0.000021,0.000028
F,0.000068,0.000495,0.000021,0.000076
KO,0.000021,0.000021,0.000072,0.000036
VZ,0.000028,0.000076,0.000036,0.000209


In [10]:
# For this example we will use an equal weight scheme
# In practice this is determined by a portfolio optimization algorithm
weights = np.ones(len(tickers)) / len(tickers)

weights

array([0.25, 0.25, 0.25, 0.25])

### Portfolio Metrics

In [11]:
# Portfolio return
portfolio_return = weights.T @ expected_returns

f"{round(portfolio_return*100,2)}%"

'0.07%'

In [12]:
# Portfolio volatility
portfolio_volatility = np.sqrt(weights.T @ covariance_matrix @ weights)

f"{round(portfolio_volatility*100,2)}%"

'0.95%'

In [13]:
# Portfolio sharpe
portfolio_sharpe = portfolio_return / portfolio_volatility

f"{round(portfolio_sharpe,2)}"

'0.07'

### Annualized metrics

In [14]:
# Annualize
TRADING_DAYS = 252

In [15]:
# Annual portfolio return
annual_portfolio_return = portfolio_return * TRADING_DAYS

f"{round(annual_portfolio_return*100,2)}%"

'16.95%'

In [16]:
# Annual portfolio volatility
annual_portfolio_volatility = portfolio_volatility * np.sqrt(TRADING_DAYS)

f"{round(annual_portfolio_volatility*100,2)}%"

'15.04%'

In [17]:
# Annual portfolio sharpe
annual_portfolio_sharpe = annual_portfolio_return / annual_portfolio_volatility

f"{round(annual_portfolio_sharpe,2)}"

'1.13'